In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE
from sklearn import preprocessing

In [70]:
acquired_dict = {
    'tr':'tr',
    'fa':'fa',
    'dr':'dr',
    "Traded": "tr",
    "Free Agency": "fa",
    "Amateur Draft": "dr",
    "Amateur Free Agent": "fa",
    "Waivers": "tr",
    "Purchased":"tr",
    "Rule 5 Draft": "dr",
    "Expansion Draft": "dr",
    "Conditional Deal": "tr",
    "Amateur Draft--no sign": "dr",
    "MinorLg Draft": "dr",
    "Rune 5 returned": "tr"
}

def inflation_calc(row):
    inf_dict = {
     2017: 1.0,
     2016: 1.021299290023666,
     2015: 1.0341874211554445,
     2014: 1.0354149770208165,
     2013: 1.0522113523096537,
     2012: 1.0676237183898534,
     2011: 1.089717656786951,
     2010: 1.1241149062626115,
     2009: 1.1425534989302544,
     2008: 1.1384885486964882,
     2007: 1.1822013870802828,
     2006: 1.215873015873016,
     2005: 1.2550947260624679,
     2004: 1.297617787188989,
     2003: 1.3324635790389214,
     2002: 1.3626862352679565,
     2001: 1.3843112893206078,
     2000: 1.4234610917537749
    }
    return int(row['salary']*inf_dict[row['year']])

def fixtm(t):
    if t == '2TM' or t == '3TM' or t == '4TM':
        return 'multiple'
    elif t == 'TBD':
        return 'TBR'
    elif t == 'MON':
        return "WSN"
    elif t == 'ANA':
        return 'LAA'
    elif t == 'FLA':
        return 'MIA'
    else: return t

In [108]:
def train_and_test(cutoff = 1000000):
    train_X,train_y,test_X,test_y = load_and_split_data()
    
    lr = LinearRegression()

    lr.fit(train_X, train_y)

    preds = lr.predict(test_X)

    error = np.sqrt(MSE(test_y,preds))
    
    return "The error is a factor of {0}".format(round(10**error,2))

In [111]:
def load_data():
    train = pd.read_pickle('batting_00_16.pkl')
    test = pd.read_pickle('batting_17.pkl')
    return train,test

In [159]:
def engineer_features(df):
    df = df.reset_index()
    #adjust team names
    df['tm'] = df['tm'].apply(fixtm)
    #drop position summary (too many classes), log_sal (unscaled by inflation), rk (same as index)
    df.drop(['pos\xa0summary','log_sal','rk','index'],axis=1,inplace=True)
    #map values in acquired to 3 classes
    df['acquired'] = df['acquired'].map(acquired_dict)
    
    #adjust salary for inflation and take the log-10 for target column
    df['adj_salary'] = df.apply(inflation_calc,axis=1)
    df['log10_adj'] = np.log10(df['adj_salary'])
    
    #get dummy variables for team, hand, and acquired columns
    df = pd.get_dummies(df,columns = ['acquired','bat_hand','tm']).drop(['tm_multiple','bat_hand_rhb','acquired_tr'],axis=1)
    #filter datasets for only batters with more than 200 plate appearances in season
    df = df[df.pa>200]

    return df

    

def rescale_numeric(df):
    
    cols = [ 'g',
         'pa',
         'rbat',
         'rbaser',
         'rdp',
         'rfield',
         'rpos',
         'raa',
         'waa',
         'rrep',
         'rar',
         'war',
         'waawl%',
         '162wl%',
         'owar',
         'dwar',
         'orar',
         'year',]
    
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(df[cols])
    df_normalized = pd.DataFrame(np_scaled)
    df_normalized.columns = cols
    df[cols] = df_normalized 
    return df

In [116]:
def load_and_split_data():
    train,test = load_data()
    
    train = engineer_features(train)
    test = engineer_features(test)
    train = rescale_numeric(train)
    test = rescale_numeric(test)
    
    train_y = train['log10_adj']
    test_y = test['log10_adj']
    train_X = train.drop(['name','age','log10_adj'],axis=1)
    test_X = test.drop(['name','age','log10_adj'],axis=1)
    
    return train_X, train_y, test_X, test_y

In [154]:
a,b = load_data()

In [155]:
a.shape

(13464, 27)

In [156]:
a['g'].isnull().sum()

0

In [157]:
engineer_features((a)).shape

(5395, 57)

In [160]:
rescale_numeric(engineer_features(a))['g'].isnull().sum()

3175

In [147]:
a[a.pa>200].shape

(5395, 27)

In [146]:
a

,rk,name,age,tm,g,pa,rbat,rbaser,rdp,rfield,...,162wl%,owar,dwar,orar,salary,acquired,pos summary,bat_hand,year,log_sal
0,1,Jeff Abbott,27,CHW,80,242,-7,1,1,-6,...,0.493,0.3,-0.6,2,255000.0,Amateur Draft,879/D,rhb,2000,5.406540
1,2,Kurt Abbott,31,NYM,79,173,-9,0,0,-2,...,0.495,-0.2,0.0,-1,500000.0,Free Agency,64/58,rhb,2000,5.698970
2,3,Paul Abbott,32,SEA,2,6,0,0,0,0,...,0.501,0.1,0.0,1,285000.0,Free Agency,1,rhb,2000,5.454845
3,4,Bobby Abreu,26,PHI,154,680,38,-2,1,15,...,0.527,4.9,0.8,52,2933333.0,Traded,*9,lhb,2000,6.467361
4,5,Juan Acevedo,30,MIL,60,2,0,0,0,0,...,0.500,0.0,0.0,0,612500.0,Traded,1,rhb,2000,5.787106
5,6,Terry Adams,27,LAD,63,3,-1,0,0,0,...,0.500,0.0,0.0,-1,1400000.0,Traded,1,rhb,2000,6.146128
6,7,Benny Agbayani,28,NYM,119,415,13,0,0,0,...,0.506,2.1,-0.5,23,220000.0,Amateur Draft,*79/8D,rhb,2000,5.342423
7,8,Rick Aguilera,38,CHC,50,0,0,0,0,0,...,0.500,0.0,0.0,0,3500000.0,Traded,1,rhb,2000,6.544068
8,9,Israel Alcantara,27,BOS,21,48,2,1,0,0,...,0.501,0.3,-0.2,3,200000.0,Free Agency,/D937,rhb,2000,5.301030
9,10,Scott Aldred,32,PHI,23,0,0,0,0,0,...,0.500,0.0,0.0,0,600000.0,Traded,1,lhb,2000,5.778151
